# Loading The Datasets

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

# Base directory for Data_Tensorflow
DATA_TENSORFLOW_DIR = '/content/drive/MyDrive/Colab Notebooks/Datasets/Phase 3 Data/Data_Tensorflow'

# Define paths for each NumPy array
X_train_path = os.path.join(DATA_TENSORFLOW_DIR, 'X_training_f2_p2.npy')
Y_train_path = os.path.join(DATA_TENSORFLOW_DIR, 'Y_training_f2_p2.npy')

X_train_dev_path = os.path.join(DATA_TENSORFLOW_DIR, 'X_training_dev_f2_p2.npy')
Y_train_dev_path = os.path.join(DATA_TENSORFLOW_DIR, 'Y_training_dev_f2_p2.npy')

X_dev_path = os.path.join(DATA_TENSORFLOW_DIR, 'X_dev_f2_p2.npy')
Y_dev_path = os.path.join(DATA_TENSORFLOW_DIR, 'Y_dev_f2_p2.npy')

X_test_path = os.path.join(DATA_TENSORFLOW_DIR, 'X_test_f2_p2.npy')
Y_test_path = os.path.join(DATA_TENSORFLOW_DIR, 'Y_test_f2_p2.npy')

X_train_empirical_path = os.path.join(DATA_TENSORFLOW_DIR, 'X_train_empirical.npy')

Y_train_empirical_path = os.path.join(DATA_TENSORFLOW_DIR, 'Y_train_empirical.npy')


In [3]:
import numpy as np

# Load training data
X_train = np.load(X_train_path)
Y_train = np.load(Y_train_path)

# Load training_dev data
X_train_dev = np.load(X_train_dev_path)
Y_train_dev = np.load(Y_train_dev_path)

# Load dev data
X_dev = np.load(X_dev_path)
Y_dev = np.load(Y_dev_path)

# Load test data
X_test = np.load(X_test_path)
Y_test = np.load(Y_test_path)

#load empirical data
X_train_empirical = np.load(X_train_empirical_path)
Y_train_empirical = np.load(Y_train_empirical_path)

print("✅ Successfully loaded all NumPy arrays including empirical data.")


✅ Successfully loaded all NumPy arrays including empirical data.


In [4]:
print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")
print(f"X_train_dev shape: {X_train_dev.shape}, Y_train_dev shape: {Y_train_dev.shape}")
print(f"X_dev shape: {X_dev.shape}, Y_dev shape: {Y_dev.shape}")
print(f"X_test shape: {X_test.shape}, Y_test shape: {Y_test.shape}")

print(f"Data types - X: {X_train.dtype}, Y: {Y_train.dtype}")

print(f"X_train_empirical shape: {X_train_empirical.shape}, Y_train_empirical shape: {Y_train_empirical.shape}")


X_train shape: (129670, 119), Y_train shape: (129670, 4)
X_train_dev shape: (7500, 119), Y_train_dev shape: (7500, 4)
X_dev shape: (7500, 119), Y_dev shape: (7500, 4)
X_test shape: (7500, 119), Y_test shape: (7500, 4)
Data types - X: float32, Y: float32
X_train_empirical shape: (48649, 119), Y_train_empirical shape: (48649, 4)


# Feature Scaling

In [5]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform
X_train_scaled = scaler.fit_transform(X_train)
X_train_dev_scaled = scaler.transform(X_train_dev)
X_dev_scaled = scaler.transform(X_dev)
X_test_scaled = scaler.transform(X_test)
X_train_empirical_scaled = scaler.transform(X_train_empirical)

print("✅ Feature scaling completed.")


✅ Feature scaling completed.


In [6]:
X_train_scaled = X_train_scaled.astype(np.float32)
X_train_dev_scaled = X_train_dev_scaled.astype(np.float32)
X_dev_scaled = X_dev_scaled.astype(np.float32)
X_test_scaled = X_test_scaled.astype(np.float32)
X_train_empirical_scaled = X_train_empirical_scaled.astype(np.float32)

Y_train = Y_train.astype(np.float32)
Y_train_dev = Y_train_dev.astype(np.float32)
Y_dev = Y_dev.astype(np.float32)
Y_test = Y_test.astype(np.float32)
Y_train_empirical=Y_train_empirical.astype(np.float32)

print("✅ Data type conversion completed.")


✅ Data type conversion completed.


In [7]:
print(f"X_train shape: {X_train_scaled.shape}, Y_train shape: {Y_train.shape}")
print(f"X_train_dev shape: {X_train_dev_scaled.shape}, Y_train_dev shape: {Y_train_dev.shape}")
print(f"X_dev shape: {X_dev_scaled.shape}, Y_dev shape: {Y_dev.shape}")
print(f"X_test shape: {X_test_scaled.shape}, Y_test shape: {Y_test.shape}")
print(f"X_train_empirical shape: {X_train_empirical_scaled.shape}, Y_train_empirical shape: {Y_train_empirical.shape}")

print(f"Data types - X: {X_train.dtype}, Y: {Y_train.dtype}")

X_train shape: (129670, 119), Y_train shape: (129670, 4)
X_train_dev shape: (7500, 119), Y_train_dev shape: (7500, 4)
X_dev shape: (7500, 119), Y_dev shape: (7500, 4)
X_test shape: (7500, 119), Y_test shape: (7500, 4)
X_train_empirical shape: (48649, 119), Y_train_empirical shape: (48649, 4)
Data types - X: float32, Y: float32


# Building The Model

In [8]:
import tensorflow as tf

# Number of hidden layers
num_hidden_layers = 40

# Detecting the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# Creating the distribution strategy
strategy = tf.distribute.TPUStrategy(tpu)

# Building the model using a list of layers
model_layers = []

for _ in range(num_hidden_layers):
    # Add Dense layer without activation and without bias
    model_layers.append(
        tf.keras.layers.Dense(
            units=256,
            # kernel_regularizer=tf.keras.regularizers.l2(0.0000001),
            kernel_initializer='glorot_uniform',
            use_bias=False  # Disable bias when using BatchNorm
        )
    )
    # Add Batch Normalization
    model_layers.append(tf.keras.layers.BatchNormalization())
    # Add SiLU Activation
    model_layers.append(tf.keras.layers.Activation('silu'))

# Add the final classification layer
model_layers.append(
    tf.keras.layers.Dense(
        units=4,
        activation='softmax',
        # kernel_regularizer=tf.keras.regularizers.l2(0.0000001),
        kernel_initializer='glorot_uniform'
    )
)

# Building the model
model = tf.keras.Sequential(model_layers)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6),
    loss='categorical_crossentropy',
    metrics=[tf.keras.metrics.CategoricalAccuracy()]
)

# Training the model
history = model.fit(X_train_empirical_scaled, Y_train_empirical, batch_size=32, epochs=40)


Epoch 1/40
1521/1521 [==============================] - 59s 30ms/step - loss: 1.3598 - categorical_accuracy: 0.4101
Epoch 2/40
1521/1521 [==============================] - 41s 27ms/step - loss: 1.2772 - categorical_accuracy: 0.5157
Epoch 3/40
1521/1521 [==============================] - 43s 28ms/step - loss: 1.2226 - categorical_accuracy: 0.5448
Epoch 4/40
1521/1521 [==============================] - 40s 26ms/step - loss: 1.1742 - categorical_accuracy: 0.5654
Epoch 5/40
1521/1521 [==============================] - 40s 26ms/step - loss: 1.1405 - categorical_accuracy: 0.5792
Epoch 6/40
1521/1521 [==============================] - 40s 27ms/step - loss: 1.1052 - categorical_accuracy: 0.5913
Epoch 7/40
1521/1521 [==============================] - 39s 26ms/step - loss: 1.0785 - categorical_accuracy: 0.5991
Epoch 8/40
1521/1521 [==============================] - 40s 26ms/step - loss: 1.0505 - categorical_accuracy: 0.6093
Epoch 9/40
1521/1521 [==============================] - 40s 26ms/step - 

# Error Analysis

***1.Categorical Accuracy***

In [9]:
training_categorical_accuracy = history.history['categorical_accuracy'][-1]
training_dev_accuracy=model.evaluate(X_train_dev_scaled,Y_train_dev)
Dev_accuracy=model.evaluate(X_dev_scaled,Y_dev)
Test_accuracy=model.evaluate(X_test_scaled,Y_test)

#Different training distributions (Empirical & Mixed)
Train_empirical_accuracy=model.evaluate(X_train_empirical_scaled,Y_train_empirical)

Train_mixed_accuracy=model.evaluate(X_train_scaled,Y_train)


print(f"Training Categorical Accuracy: {training_categorical_accuracy}")
print(f"Training Dev accuracy: {training_dev_accuracy[1]}")
print(f"Dev accuracy: {Dev_accuracy[1]}")
print(f"Test accuracy: {Test_accuracy[1]}")
print(f"Empirical accuracy: {Train_empirical_accuracy[1]}")
print(f'Mixed data accuracy: {Train_mixed_accuracy[1]}')

4053/4053 [==============================] - 27s 7ms/step - loss: 1.9070 - categorical_accuracy: 0.5205
Training Categorical Accuracy: 0.6585541367530823
Training Dev accuracy: 0.5225333571434021
Dev accuracy: 0.6565333604812622
Test accuracy: 0.6738666892051697
Empirical accuracy: 0.6666529774665833
Mixed data accuracy: 0.5205367207527161


***2.Error***

In [10]:
Train_Error=1-training_categorical_accuracy
print(f"Training Error: {Train_Error}")

Train_Dev_Error=1-training_dev_accuracy[1]
print(f"Training Dev Error: {Train_Dev_Error}")

Dev_Error=1-Dev_accuracy[1]
print(f"Dev Error: {Dev_Error}")

Test_Error=1-Test_accuracy[1]
print(f"Test_Error: {Test_Error}")

Train_empirical_Error=1-Train_empirical_accuracy[1]
print(f"Distribution II_Empirical Error: {Train_empirical_Error}")

Train_mixed_Error=1-Train_mixed_accuracy[1]
print(f"Distribution I_Mixed Error: {Train_mixed_Error}")


Training Error: 0.3414458632469177
Training Dev Error: 0.4774666428565979
Dev Error: 0.3434666395187378
Test_Error: 0.3261333107948303
Distribution II_Empirical Error: 0.33334702253341675
Distribution I_Mixed Error: 0.47946327924728394


# Save the model

In [12]:
model.save('/content/drive/MyDrive/Colab Notebooks/Main_code/Model_v4.keras')